In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./4.model/'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:

    # = = = = = = = = = = = = = = =
    
    output_correct, output_error = pd.DataFrame(), pd.DataFrame()
    
    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./4.model/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Make No'] = input_['Make No'].astype(int)
    input_['Model No'] = input_['Model No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)
    
    print(f'总数量：{len(input_)}')
    print()
    
    # = = = = = = = = = = = = = = =
    
    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)
    
    # = = = = = = = = = = = = = = =
    
    def crawler():
        global output_correct, output_error
    
        while not work.empty():
            a = work.get_nowait()
    
            # = = = = = = = = = = = = = = =
    
            crawler_status = 'error'
    
            # = = = = = = = = = = = = = = =
    
            data = {'cssClass': 'form-control',
                    'dropdownType': '3',
                    'selectedValue': '',
                    'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                                          'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                                          'ModelID': f'''{input_.loc[a, 'Model Code']}''',
                                          'SkinID': '1',
                                          'Year': f'''{input_.loc[a, 'Year']}'''}}
    
            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                                             data=json.dumps(data),
                                             headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        
                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                json_ = resp.json()

                # = = = = = = = = = = = = = = =
                        
                soup = BeautifulSoup(json_['d'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_type_code = html.xpath('//select[@id="parttypeDropdown"]/option/@value')[1:]

                # = = = = = = = = = = = = = = =

                list_type = html.xpath('//select[@id="parttypeDropdown"]/option/text()')[1:]
    
                # = = = = = = = = = = = = = = =
    
                if not list_type:
                    raise
    
                # = = = = = = = = = = = = = = =
    
                df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                        'Make': input_.loc[a, 'Make'],
                                        'Make Code': input_.loc[a, 'Make Code'],
                                        'Model No': input_.loc[a, 'Model No'],
                                        'Model': input_.loc[a, 'Model'],
                                        'Model Code': input_.loc[a, 'Model Code'],
                                        'Year': input_.loc[a, 'Year'],
                                        'Type No': [i+1 for i in range(len(list_type))],
                                        'Type': list_type,
                                        'Type Code': list_type_code,
                                        'Type Url': [f'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q={type_}' for type_ in list_type]})
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
    
                # = = = = = = = = = = = = = = =
    
                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                         'Make': input_.loc[a, 'Make'],
                                         'Make Code': input_.loc[a, 'Make Code'],
                                         'Model No': input_.loc[a, 'Model No'],
                                         'Model': input_.loc[a, 'Model'],
                                         'Model Code': input_.loc[a, 'Model Code'],
                                         'Year': input_.loc[a, 'Year']}])
    
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
            # = = = = = = = = = = = = = = =
    
            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()
    
            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Model No']}.{input_.loc[a, 'Model']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
    # = = = = = = = = = = = = = = =
    
    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)
    
    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Make No', 'Model No', 'Year', 'Type No'],
                                                    ascending=[True, True, False, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./5.type_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Make No', 'Model No', 'Year'],
                                                ascending=[True, True, False],
                                                ignore_index=True)
        output_error.to_excel(f'''./5.type_1/0.error/{file.removesuffix('.xlsx')}-type_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
print('Done ~')

总数量：1

[状态：ok，尝试次数：1] - 100.00% - make_102.Land Rover-type_1_error > 15.Range Rover Sport > 2010
[剩余数量：0] - [当前时间：13:59:03]

输出ing...

Done ~
